In [1]:
import tensorflow as tf

2023-12-21 17:43:18.958170: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-21 17:43:19.137891: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 17:43:19.137932: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 17:43:19.156549: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 17:43:19.246070: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-21 17:43:19.247760: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
data_path = "dataset"
train_data = tf.keras.utils.audio_dataset_from_directory(
    data_path,
    batch_size=8,
    shuffle=True,
    validation_split=0.2,
    subset="training",
    output_sequence_length=48000,
    ragged=False,
    label_mode="categorical",
    labels="inferred",
    sampling_rate=None,
    seed=60

)
validation_data = tf.keras.utils.audio_dataset_from_directory(
    data_path,
    batch_size=8,
    shuffle=True,
    validation_split=0.2,
    subset="validation",
    output_sequence_length=48000,
    ragged=False,
    label_mode="categorical",
    labels="inferred",
    sampling_rate=None,
    seed=60
)


Found 1619 files belonging to 17 classes.
Using 1296 files for training.
Found 1619 files belonging to 17 classes.
Using 323 files for validation.


In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(32, kernel_size=80, activation='relu', strides=16, input_shape=(48000, 1)),
    tf.keras.layers.MaxPooling1D(4),
    tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(4),
    tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(4),
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(4),
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(4),
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(17, activation='softmax')
])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2996, 32)          2592      
                                                                 
 max_pooling1d (MaxPooling1  (None, 749, 32)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 747, 32)           3104      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 186, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_2 (Conv1D)           (None, 184, 32)           3104      
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 46, 32)            0

In [5]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [6]:
model.fit(train_data, validation_data=validation_data, epochs=100)

Epoch 1/100


162/162 [==============================] - 4s 19ms/step - loss: 2.5264 - accuracy: 0.1343 - val_loss: 2.2413 - val_accuracy: 0.2508
Epoch 2/100
162/162 [==============================] - 3s 18ms/step - loss: 1.9585 - accuracy: 0.3426 - val_loss: 1.6458 - val_accuracy: 0.4861
Epoch 3/100
162/162 [==============================] - 3s 18ms/step - loss: 1.3519 - accuracy: 0.5231 - val_loss: 1.4081 - val_accuracy: 0.5418
Epoch 4/100
162/162 [==============================] - 3s 18ms/step - loss: 1.0346 - accuracy: 0.6427 - val_loss: 1.2863 - val_accuracy: 0.5635
Epoch 5/100
162/162 [==============================] - 3s 18ms/step - loss: 0.8912 - accuracy: 0.7045 - val_loss: 0.8574 - val_accuracy: 0.6966
Epoch 6/100
162/162 [==============================] - 3s 18ms/step - loss: 0.6667 - accuracy: 0.7747 - val_loss: 0.8050 - val_accuracy: 0.7647
Epoch 7/100
162/162 [==============================] - 3s 18ms/step - loss: 0.5333 - accuracy: 0.8341 - val_loss: 0.7380 - val_accuracy: 0.7802
Epoc

In [7]:
model.save('audio_friends.h5')

/home/mohammad/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
